## Import SAS & Open Source Packages
Postprocessing done in JupyterLab.

In [ ]:
#Imports
import threading
import time
import websocket
import json
import numpy as np
import base64
import cv2
import esppy

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cv2.imshow('frame',gray)

In [ ]:
import tensorflow as tf

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

In [ ]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True)) as sess:
    print (sess.run(c))

In [ ]:
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0, cv2.CAP_V4L)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
print(cv2.getBuildInformation())

## Build SAS Event Stream Processing Pipeline

In [ ]:
#Connect to ESP and create ESP project
esp = esppy.ESP(hostname='http://localhost:9900')
esp_project = esp.create_project('object_detection', n_threads=10)
esp_project.pubsub = 'manual'
esp_project.add_continuous_query('contquery')

# Window: Video Capture
vid_capture = esp.SourceWindow(schema=('id*:int64', 'image:blob'),
index_type='empty', insert_only=True)
vid_capture.pubsub = True
esp_project.windows['w_input_image'] = vid_capture

# Window: Video Resize
vid_capture_resize = esp.CalculateWindow(algorithm='ImageProcessing', 
                                         name='resized', 
                                         function='resize',
                                         height=416, 
                                         width=416, 
                                         input_map=dict(imageInput='image'), 
                                         output_map=dict(imageOutput='_image_'))
vid_capture_resize.schema_string = 'id*:int64,image:blob,_image_:blob'
esp_project.windows['w_resize_image'] = vid_capture_resize

# Window: Model Reader
model_reader = esp.ModelReaderWindow()
esp_project.windows['w_read_model'] = model_reader

# Window: Model Request
model_request = esp.SourceWindow(schema=('req_id*:int64', 'req_key:string', 'req_val:string'),index_type='empty', insert_only=True)
esp_project.windows['w_request_model'] = model_request

# Window: Model Score
model_score = esp.ScoreWindow()
model_score.pubsub = True
model_score.add_offline_model(model_type='astore')
def score_window_fields(number_objects):
    _field = "id*:int64,image:blob,_image_:blob,_nObjects_:double,"
    for obj in range(0,number_objects):
        _field += "_Object" + str(obj) + "_:string,"
        _field += "_P_Object" + str(obj) + "_:double,"
        _field += "_Object" + str(obj) + "_x:double,"
        _field += "_Object" + str(obj) + "_y:double,"
        _field += "_Object" + str(obj) + "_width:double,"
        _field += "_Object" + str(obj) + "_height:double,"
    return _field[:-1]
model_score.schema_string = score_window_fields(20)
esp_project.windows['w_score_image'] = model_score

# Connections
vid_capture.add_target(vid_capture_resize, role='data')
vid_capture_resize.add_target(model_score, role='data')
model_request.add_target(model_reader, role='request')
model_reader.add_target(model_score, role='model')

# Load Project
esp.load_project(esp_project)

# Publisher: Send Model to Scoring Window
pub = model_request.create_publisher(blocksize=1, rate=0, pause=0, dateformat='%Y%dT%H:%M:%S.%f', opcode='insert', format='csv')
pub.send('i,n,1,"usegpuesp","1"\n')
pub.send('i,n,2,"ndevices","1"\n')
pub.send('i,n,3,"action","load"\n')
pub.send('i,n,4,"type","astore"\n')
pub.send('i,n,5,"reference","/data/notebooks/SAS_DeepLearning/Real Time Object Detection/Tiny-Yolov2.astore"\n')
pub.send('i,n,6,,\n')
pub.close()

# Publisher: Send Video
pub = vid_capture.create_publisher(blocksize=1, rate=0, pause=0, opcode='insert', format='csv')

# Display project
esppy.options.display.image_scale = 0.8
esp_project

## Define Video-Publisher

In [ ]:
# Class to publish videos from camera to ESP
class video_pub():
    def __init__(self, pub, fps):
        self.cap = cv2.VideoCapture(0)
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        self.cap.set(cv2.CAP_PROP_FOURCC, fourcc)
        self.cap.set(3,1280)
        self.cap.set(4,720)
        self.pub = pub
        self.FPS = fps
        threading.Thread(target=self.stream, daemon=True).start()
        print('Publisher started!')
        
    def stream(self):
        prev = 0
        while True:
            time_elapsed = time.time() - prev
            if time_elapsed > 1./self.FPS:
                prev = time.time()
                ret, frame = self.cap.read()
                frame = cv2.flip(frame, 1)
                _, buffer = cv2.imencode('.jpg', frame)
                encoded_string = base64.b64encode(buffer)
                strToSend = 'i, n, ' + str(int(time.time()*100)) + ',' + encoded_string.decode() + ',' + '\n'
                self.pub.send(strToSend)

## Define Video-Subscriber to obtain scored images

In [ ]:
#Class to subscribe to ESP to receive and highlight scored images
class video_sub():
    def __init__(self):
        self.ws = websocket.WebSocketApp("ws://localhost:9900/SASESP/subscribers/object_detection/contquery/w_score_image/?format=json&mode=streaming&pagesize=1",
                                 on_message = self.on_message,
                                 on_error = self.on_error,
                                 on_close = self.on_close)
        self.ws.on_open = self.on_open
        self.frame = None
        threading.Thread(target=self.ws.run_forever, daemon=True).start()
        print('Subscriber Started!')
        return
        
    def highlightImage(self, data):
        #object_list = ['Human face', 'Shirt', 'Person', 'Dress', 'Fashion accessory', 'Glasses', 'Handbag', 'Hat', 'Jewelry', 'Rifle', 'Trousers', 'Skirt', 'Weapon']
        object_list = ['Human face', 'Shirt', 'Fashion accessory', 'Jewelry']#13
        color_palette = [
        (0,64,255), #red
        (0,191,255), #orange
        (0,255,255), #yellow
        (0,255,64), #green
        (255,255,0), #blue
        (250,0,250), #pink
        (250,0,125), #purple
        (167,250,0), #turquoise
        (255,200,0), #light-blue
        (255,100,0), #dark-blue
        (0,255,100), #light-green
        (155,0,255), #pink
        (255,170,0) #blue
        ]
        #BGR Colorcodes
        obj_colors = {}
        i = 0
        for _object in object_list:
            obj_colors[_object] = color_palette[i]
            i += 1

        row = data['events'][0]['event']
        numberOfObjects = data['events'][0]['event']['_nObjects_']
        imageBufferBase64 = data['events'][0]['event']['image']['image']

        nparr = np.frombuffer(base64.b64decode(imageBufferBase64), dtype=np.uint8)
        frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        image_h, image_w,_ = frame.shape
        for i in range(0, int(float(numberOfObjects))):
            obj = row['_Object' + str(i) + '_']
            prob = float(row['_P_Object' + str(i) + '_'])
            probability = " (" + str(round(prob * 100, 2)) + "%)"
            x = float(row['_Object' + str(i) + '_x'])
            y = float(row['_Object' + str(i) + '_y'])
            width = float(row['_Object' + str(i) + '_width'])
            height = float(row['_Object' + str(i) + '_height'])
            x1 = int(image_w * (x - width / 2))
            y1 = int(image_h * (y - height/ 2))
            x2 = int(image_w * (x + width / 2))
            y2 = int(image_h * (y + height/ 2))
            if obj in obj_colors:
                bbox_color = obj_colors[obj]
                border_offset = 3
                cv2.rectangle(frame,(x1,y1),(x2,y2),bbox_color,1)
                (label_width, label_height), baseline = cv2.getTextSize(obj + probability, cv2.FONT_HERSHEY_DUPLEX, 0.4, 1)
                cv2.rectangle(frame,(x1,y1),(x1+label_width+10,y1-label_height-border_offset-10),bbox_color,-1)
                cv2.putText(frame, obj + probability, (x1+5, y1-border_offset-5), cv2.FONT_HERSHEY_DUPLEX, 0.4, (0, 0, 0), 1,
                    cv2.LINE_AA)
        return frame

    def on_message(self, message):
        try:
            data = json.loads(message)
            self.frame = self.highlightImage(data)
        except:
            None

    def on_error(self, error):
        print(error)

    def on_close(self):
        print("### closed ###")

    def on_open(self):
        print('open')

## Start the Video-Publisher & -Subscriber

In [ ]:
pub = video_pub(pub, 25)
sub = video_sub()

## Display scored images

In [ ]:
#Displays scored frames
while(True):
    # Capture frame-by-frame
    frame = sub.frame

    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()